In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib
from collections import Counter
height, width = 10, 10

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.05)
label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 2568, 62: 282, 83: 269, 86: 266, 47: 264, 20: 261, 50: 260, 92: 257, 1: 257, 60: 255, 14: 255, 59: 255, 29: 254, 34: 253, 49: 253, 8: 252, 72: 252, 55: 252, 79: 251, 12: 248, 87: 248, 32: 248, 70: 247, 23: 247, 24: 246, 56: 245, 4: 245, 69: 244, 52: 244, 95: 244, 63: 244, 80: 243, 76: 242, 37: 242, 21: 241, 65: 240, 18: 240, 93: 240, 90: 240, 35: 240, 58: 239, 22: 238, 9: 238, 16: 238, 54: 238, 64: 238, 45: 238, 40: 236, 11: 236, 6: 236, 51: 235, 26: 235, 15: 235, 17: 235, 74: 234, 44: 234, 66: 233, 71: 232, 31: 231, 33: 231, 39: 231, 36: 231, 38: 230, 68: 230, 13: 229, 42: 228, 77: 228, 57: 228, 41: 227, 84: 227, 82: 226, 3: 226, 75: 225, 7: 225, 10: 224, 91: 224, 67: 224, 48: 223, 88: 223, 89: 221, 19: 221, 85: 220, 53: 220, 43: 220, 61: 219, 28: 218, 73: 218, 78: 215, 46: 214, 27: 214, 81: 212, 25: 211, 94: 210, 5: 209, 2: 206, 30: 199})


In [3]:
X = data.iloc[:, 1:].astype("float64")
y = data.iloc[:, 0].astype("float64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/random_forest_model_10_x_10.pkl')

['../artifacts/random_forest_model_10_x_10.pkl']

In [5]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 100)
Train Accuracy: 97.8200%
Test Accuracy: 92.5000%
F1 Score: 91.7636%
Recall: 92.5000%


In [6]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

In [7]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/random_forest_model_hog_10_x_10.pkl')

['../artifacts/random_forest_model_hog_10_x_10.pkl']

In [8]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 225)
Train Accuracy: 97.7800%
Test Accuracy: 95.2400%
F1 Score: 94.4890%
Recall: 95.2400%


In [9]:
import cv2

def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

In [10]:
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/random_forest_model_sift_10_x_10.pkl')

['../artifacts/random_forest_model_sift_10_x_10.pkl']

In [12]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 128)
Train Accuracy: 10.1250%
Test Accuracy: 10.8600%
F1 Score: 2.1277%
Recall: 10.8600%
